<a href="https://colab.research.google.com/github/junyamahira/ReinforcementLearning_Prac/blob/master/DQN_MountainCar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# keras-rlのインストール

## git clone 
- これかpipのどちらかを実行すればよい (pipがおすすめ)
- keras-rlの中に入り、setup.pyを実行する

In [0]:
!git clone https://github.com/keras-rl/keras-rl.git

Cloning into 'keras-rl'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 1710 (delta 3), reused 7 (delta 2), pack-reused 1698
Receiving objects: 100% (1710/1710), 1.38 MiB | 19.35 MiB/s, done.
Resolving deltas: 100% (1056/1056), done.


In [0]:
!ls keras-rl

assets		 examples	    mkdocs.yml	rl	   tests
CONTRIBUTING.md  ISSUE_TEMPLATE.md  pytest.ini	setup.cfg  utils
docs		 LICENSE	    README.md	setup.py


In [0]:
!cd keras-rl
!ls

keras-rl  sample_data


## pip

In [8]:
pip install keras-rl

    100% |████████████████████████████████| 40kB 5.2MB/s 
  Stored in directory: /root/.cache/pip/wheels/7d/4d/84/9254c9f2e8f51865cb0dac8e79da85330c735551d31f73c894
Successfully built keras-rl


# その他のimport

In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam
import gym
from gym import wrappers
from rl.agents.dqn import DQNAgent
from rl.policy import EpsGreedyQPolicy
from rl.memory import SequentialMemory

## visiual display

In [0]:
#?
!apt-get -qq -y install xvfb freeglut3-dev ffmpeg> /dev/null

!pip -q install gym
!pip -q install pyglet
!pip -q install pyopengl
!pip -q install pyvirtualdisplay

In [0]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1024, 768))
display.start()
import os
os.environ["DISPLAY"] = ":" + str(display.display) + "." + str(display.screen)

#　

In [32]:
env = gym.make("MountainCar-v0")
env = wrappers.Monitor(env, './', force=True)
nb_actions = env.action_space.n

NameError: ignored

# モデル

In [13]:
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))

Instructions for updating:
Colocations handled automatically by placer.


In [19]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 2)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                48        
_________________________________________________________________
activation_1 (Activation)    (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                272       
_________________________________________________________________
activation_2 (Activation)    (None, 16)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 16)                272       
_________________________________________________________________
activation_3 (Activation)    (None, 16)                0         
__________

In [14]:
memory = SequentialMemory(limit=50000, window_length=1)
# 方策
policy = EpsGreedyQPolicy(eps=0.001)
#　各パラメータをここで調整
dqn = DQNAgent(model=model, nb_actions=nb_actions,gamma=0.99, memory=memory, nb_steps_warmup=10,
               target_model_update=1e-2, policy=policy)
# 何設定してん？
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

history = dqn.fit(env, nb_steps=50000, visualize=False, verbose=2)


Training for 50000 steps ...
Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.6/dist-packages/rl/memory.py:39: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


   200/50000: episode: 1, duration: 1.446s, episode steps: 200, steps per second: 138, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean action: 1.065 [0.000, 2.000], mean observation: -0.245 [-0.868, 0.029], loss: 0.195985, mean_absolute_error: 0.650357, mean_q: -0.647298
   400/50000: episode: 2, duration: 0.620s, episode steps: 200, steps per second: 323, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean action: 0.815 [0.000, 1.000], mean observation: -0.281 [-0.829, 0.020], loss: 0.011042, mean_absolute_error: 1.581398, mean_q: -2.291042
   600/50000: episode: 3, duration: 0.621s, episode steps: 200, steps per second: 322, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean action: 0.215 [0.000, 1.000], mean observation: -0.320 [-0.761, 0.009], loss: 0.017752, mean_absolute_error: 2.685251, mean_q: -3.964835
   800/50000: episode: 4, duration: 0.636s, episode steps: 200, steps per second: 315, episode reward: -200.000, mean rew

# 実行

In [27]:
dqn.test(env, nb_episodes=1, visualize=True)

Testing for 1 episodes ...
Episode 1: reward: -84.000, steps: 84


In [31]:
keras.callbacks.History()

TypeError: ignored

# 参考
色々役に立ちそうなkerasのcallback[https://keras.io/ja/callbacks/]